<a href="https://colab.research.google.com/github/YunTaeHyeon/SmartBuildingBigDataAnalyze/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import csv
import gensim
from sklearn.preprocessing import normalize
from itertools import combinations

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# CSV 파일 불러오기
data = pd.read_csv('/content/drive/MyDrive/경진대회 DATA/train_data.csv', encoding='cp1252')
test_data = pd.read_csv('/content/drive/MyDrive/경진대회 DATA/train_data.csv', encoding='cp1252')

df = data.rename(columns={'Àç½ÇÀÎ¿ø': 'person'})
test_df = test_data.rename(columns={'Àç½ÇÀÎ¿ø': 'person'})

# dust_pm1, dust_pm_25, dust_pm_10는 중요도가 낮기 때문에 열 제거
df = df.drop(["dust_pm1", "dust_pm_25", "dust_pm_10"], axis=1)
df = df.drop(["voc" , "PIR"], axis=1)

# 상호작용 항 생성
df["temp_humi_interaction"] = df["temp"] * df["humi"]
df["temp_co2_interaction"] = df["temp"] * df["co2"]
df["humi_co2"] = df["humi"] * df["co2"]
df["temp_humi_co2_interaction"] = df["temp"] * df["humi"] * df["co2"]

df.to_csv('modified_train_data.csv', index=False)
df.to_csv('test_data.csv', index=False)

df

,regdate,temp,humi,co2,illuminance,eco2,person,temp_humi_interaction,temp_co2_interaction,humi_co2,temp_humi_co2_interaction
0,2023-01-01 0:00,21.2,34.0,414.0,0.0,2845.0,1,720.8,8776.8,14076.0,298411.2
1,2023-01-01 0:01,21.2,34.0,411.0,0.0,2845.0,1,720.8,8713.2,13974.0,296248.8
2,2023-01-01 0:02,21.2,34.0,409.0,0.0,2845.0,1,720.8,8670.8,13906.0,294807.2
3,2023-01-01 0:03,21.2,34.0,409.0,0.0,2829.0,1,720.8,8670.8,13906.0,294807.2
4,2023-01-01 0:04,21.2,34.0,415.0,0.0,2829.0,1,720.8,8798.0,14110.0,299132.0
...,...,...,...,...,...,...,...,...,...,...,...
388795,2023-01-27 23:55,17.4,17.0,414.0,1.0,452.0,0,295.8,7203.6,7038.0,122461.2
388796,2023-01-27 23:56,17.4,17.0,410.0,1.0,452.0,0,295.8,7134.0,6970.0,121278.0
388797,2023-01-27 23:57,17.4,17.0,417.0,1.0,452.0,0,295.8,7255.8,7089.0,123348.6
388798,2023-01-27 23:58,17.5,17.0,414.0,1.0,455.0,0,297.5,7245.0,7038.0,123165.0


# 랜덤포레스트로 중요도 높은 값 찾기

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# NaN 값을 가진 행 제외하기
df = df.dropna()

# regdate 열 제외하기
df = df.drop("regdate", axis=1)

# Features와 Target 분리
X = df.drop("person", axis=1)
y = df["person"]

# 랜덤 포레스트 모델 생성
rf = RandomForestRegressor()

# 모델 학습
rf.fit(X, y)

# 변수 중요도 계산
importance = rf.feature_importances_

# 중요도 출력
for i, feature in enumerate(X.columns):
    print(f"{feature}: {importance[i]}")


temp: 0.0993274469542712
humi: 0.04579147844143386
co2: 0.03550587522232136
illuminance: 0.07291455494900903
eco2: 0.06940344374200488
temp_humi_interaction: 0.0598227286903929
temp_co2_interaction: 0.5841001357352819
humi_co2: 0.018499656581630393
temp_humi_co2_interaction: 0.014634679683654561
